In [1]:
suppressMessages(suppressWarnings({
  library(Seurat)
  library(SeuratDisk)
  library(scater)
  library(cowplot)
  library(reticulate)
  library(Matrix)
  library(schard) 
  library(patchwork)
  library(ggplot2)
  library(reshape2)
  }))

In [2]:
# Ruta al archivo .h5ad
h5ad_file <- "/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/Results/datos_concatenados/adata_concatenados_10_pt.h5ad"

In [3]:
# Convertir con las capas deseadas
seurat_object <- h5ad2seurat(h5ad_file)

In [4]:
seurat_object

An object of class Seurat 
23209 features across 53542 samples within 1 assay 
Active assay: RNA (23209 features, 0 variable features)
 2 layers present: counts, data

In [4]:
# Archivo con la matriz normalizada
normalized_file <- "normalized_with_names.h5ad"
normalized_object <- h5ad2seurat(normalized_file)

In [5]:
# Extraer la matriz normalizada
normalized_matrix <- normalized_object[["RNA"]]@data

# Confirmar coincidencias de nombres
if (!all(colnames(normalized_matrix) == colnames(seurat_object[["RNA"]]@counts))) {
    stop("Los nombres de las células no coinciden entre los objetos")
}
if (!all(rownames(normalized_matrix) == rownames(seurat_object[["RNA"]]@counts))) {
    stop("Los nombres de los genes no coinciden entre los objetos")
}

# Asignar la matriz normalizada al objeto principal
seurat_object[["RNA"]]@data <- normalized_matrix

In [6]:
# Leer las matrices
connectivities <- readMM("connectivities.mtx")
distances <- readMM("distances.mtx")
# Asignar a @graphs
seurat_object@graphs[["connectivities"]] <- connectivities
seurat_object@graphs[["distances"]] <- distances

In [6]:
str(seurat_object)

Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 1
  .. ..$ RNA:Formal class 'Assay' [package "SeuratObject"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:115498997] 35 38 60 73 81 85 96 100 103 104 ...
  .. .. .. .. .. ..@ p       : int [1:53543] 0 1669 4402 6952 9086 11226 12871 15221 16837 18977 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 23209 53542
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:23209] "A1BG" "A1BG-AS1" "A1CF" "A2M" ...
  .. .. .. .. .. .. ..$ : chr [1:53542] "AAACCTGAGAAGAAGC-1_PT14" "AAACCTGAGGACAGAA-1_PT14" "AAACCTGAGTGGAGTC-1_PT14" "AAACCTGAGTGTCTCA-1_PT14" ...
  .. .. .. .. .. ..@ x       : num [1:115498997] 2 1 1 1 1 1 1 1 1 1 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:115498997] 35 

In [8]:
# Verificar si hay gráficos (graphs) en Seurat
names(seurat_object@graphs)

[1] "connectivities" "distances"

In [5]:
# Guardar el objeto Seurat en un archivo .rds
saveRDS(seurat_object, file = "/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/Results/datos_concatenados/Seurat_concatenados_10_pt.rds")

In [5]:
# Convertir con las capas deseadas
# Cargar el objeto Seurat
Atlas_TILs <- readRDS("/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/Results/firmas_transcriptomicas/Analisis_Atlas_TIL.rds")